In [640]:
import pandas as pd
import numpy as np
import ast # to convert str to list
import re

In [641]:
read_df = pd.read_csv('uncleaned_df.csv')

Dropping NA / Missing

In [642]:
"""
Drop last 4 columns which have a lot of missing values
	cols: Fiber, Vitamin A, Added Sugars, Sugars
"""
read_df.iloc[:, -4:]
read_df = read_df.iloc[:, :-4]

In [643]:
"""
1. get columns with more than 10% NA: 
	cols: 'Dietary Fiber' & 'Includes'
2. check sample of obs that have NA & look on TJ website if data is not there
or scraped poorly
	-data is not there => drop the cols
"""

# step 1
print(read_df.columns[read_df.isnull().mean() > 0.1])
read_df['Dietary Fiber'].value_counts(dropna=False) # 11
read_df['Includes'].value_counts(dropna=False) # 4

# step 2
read_df[read_df.isnull().any(axis=1)] # check which obs have any NA
read_df = read_df.drop(columns=['Dietary Fiber', 'Includes'])

Index(['Dietary Fiber', 'Includes'], dtype='object')


In [644]:
"""
Check which obs have NA vals (2):
	1. 'Saint André Triple Crème Brie'
		-does not have ANY information online => DROP obs
	2. 'Sliced New Zealand Organic Cheddar'
		-missing total sugars => Replace with ['0',''] (no sugar)
"""
read_df[read_df.isnull().any(axis=1)]
read_df = read_df[read_df['product'] != 'Saint André Triple Crème Brie'] # step 1
read_df.loc[read_df['product'] == 'Sliced New Zealand Organic Cheddar', 'Total Sugars'] = "['0','']"

In [645]:
"""
1. look @ all diff serving size values
	-most obs are '28g'
"""
read_df['serving size'].value_counts(dropna=False) # step 1
# convert '28g' (majority) to float -> 28.0
read_df.loc[read_df['serving size'].str.contains('28g'), 'serving size'] = 28.0

In [646]:
# extracting only numeric values from weight col

read_df['per weight'] = read_df['per weight'].str.extract('\/(.*)') # remove '/'
# read_df['per weight'].value_counts(dropna=False)

In [647]:
"""
looking @ other cols in list `cols`
	1. 'Potassium', 'Iron', 'Trans Fat' have lots of missing & approx all obs 0
		drop
"""
cols = ['Trans Fat', 'Total Sugars', 'Iron', 'Potassium', 'Vitamin D', 'Total Carbohydrate']
# for col in cols:
# 	print(read_df[col].value_counts(dropna=False))
read_df = read_df.drop(columns=['Potassium', 'Iron', 'Trans Fat']) #step 1

In [648]:
"""
1. convert remaining nested list (nutr) columns from str to list
	nested list is of form: ['g', '%']
2. obtain grams (not %) if poss from index 0 of nested list
"""
cols_to_list = ['Total Fat', 'Saturated Fat', 'Cholesterol', 'Sodium',
'Total Carbohydrate', 'Total Sugars', 'Protein', 'Vitamin D','Calcium']
for col in cols_to_list: # step 1
	read_df[col] = read_df[col].apply(lambda s: list(ast.literal_eval(s)))

# step 2
def get_stat(list):
	if list[0] != "": # if grams is not empty
		return list[0]
	else:
		return list[1]

for col in cols_to_list:
	read_df[col] = read_df[col].apply(get_stat)

In [649]:
"""
1. Change price to price per pound (16oz)
2. Choose grams when possible
3. Convert all values (grams or mg) to per 28 gram serving
"""

'\n1. Change price to price per pound (16oz)\n2. Choose grams when possible\n3. Convert all values (grams or mg) to per 28 gram serving\n'

In [650]:
"""
1. check to make sure price col has no non numericals
2. get name of cheese with NA price
	Name = 'Italian Lagorai Semi-Soft Cheese'
3. Look online for price & update value => $9.99
"""
read_df['price'].value_counts() # step 1
NA_price_cheese = read_df[read_df['price'] == 'Not available']['product'].item() # step 2
read_df.loc[read_df['product'] == NA_price_cheese, 'price'] = '9.99' # step 3

In [651]:
"""
Name: Vitamin D ----> DROP
0%         16
0 mcg       5
0.2 mcg     3
0.1 mcg     3
20%         1
2%          1
0.0 mcg     1
3.8 mcg     1

Name: Total Sugars ----> DROP
0 g               15
0g                 5
less than 1 g      2
1g                 2
less than 1g g     1
3g                 1
Less than 1 g      1
1 g                1
0                  1
2 g                1
3 g                1

1. standardize price to price per lb
2. drop unnecessary cols: 'per weight', 'Vitamin D', 'Total Sugars'
"""
# step 1
def get_price_per_lb(row):
	raw_weight = row['per weight']
	weight = float(re.search('(\d*\.?\d+)', raw_weight).group(1))
	raw_price = row['price']
	price = float(re.search('(\d*\.?\d+)', raw_price).group(1))
	if 'lb' in raw_weight.lower():
		price = price * (1/weight)
	elif 'oz' in raw_weight.lower():
		price = price * (16/weight)
	return price

read_df['price'] = read_df.apply(lambda row: get_price_per_lb(row), axis = 1)
read_df = read_df.drop(columns=['per weight', 'Vitamin D', 'Total Sugars'])

In [652]:
all_nutr = ['Total Fat','Saturated Fat', 'Cholesterol', 'Sodium',
'Total Carbohydrate','Protein', 'Calcium']
# checking unique vals of all nutr cols to figure out how to clean
for col in all_nutr:
	print(read_df[col].value_counts(dropna=False))

"""
Total Fat: g
Saturated Fat: g
Cholesterol: mg
Sodium: mg

Total Carbohydrate: g & less than 1 g
Protein: g & less than 1 g

Calcium: mg & %

less than 1 g -> 0 g
"""


8 g      7
5 g      4
9 g      4
6 g      3
9g       3
8g       2
7 g      2
11g      1
4.5g     1
2 g      1
10g      1
10 g     1
4.5 g    1
Name: Total Fat, dtype: int64
6 g      7
6g       6
3.5 g    5
4.5 g    2
5.0 g    2
7 g      2
5 g      1
8g       1
3g       1
4 g      1
8 g      1
1.5 g    1
3.0 g    1
Name: Saturated Fat, dtype: int64
25 mg    12
25mg      5
20 mg     3
0 mg      3
15 mg     2
20mg      2
30 mg     2
30mg      1
10 mg     1
Name: Cholesterol, dtype: int64
80 mg     3
170 mg    3
75 mg     2
210 mg    2
180 mg    2
160 mg    2
140mg     1
200mg     1
240mg     1
125mg     1
380 mg    1
190mg     1
220 mg    1
150 mg    1
75mg      1
55 mg     1
270 mg    1
160mg     1
190 mg    1
200 mg    1
230 mg    1
320mg     1
110 mg    1
Name: Sodium, dtype: int64
3 g              6
less than 1 g    3
Less than 1 g    3
0 g              3
1 g              3
2 g              2
0g               2
4 g              2
1g               2
less than 1g     1
5g               

'\nTotal Fat: g\nSaturated Fat: g\nCholesterol: mg\nSodium: mg\n\nTotal Carbohydrate: g & less than 1 g\nProtein: g & less than 1 g\n\nCalcium: mg & %\n\nless than 1 g -> 0 g\n'

3. Convert all values (grams or mg) to per 28 gram serving

In [653]:
# first extract numbers from all nutr columns
"""
1. Extract only numbers/floats from nutr cols
	a) clean easy_clean_cols (1 unique unit)
	b) clean med_clean_cols (2 unique units; less than 1 gram...)
	c) clean 'calcium` col (% and mg)
"""
easy_clean_cols = ['Total Fat', 'Saturated Fat', 'Cholesterol', 'Sodium']
med_clean_cols = ['Total Carbohydrate', 'Protein']

for col in easy_clean_cols:
	read_df[col] = read_df[col].str.extract('(\d*\.?\d+)').astype(float)

def clean_med_cols(row, col_name):
	raw_data = row[col_name]
	if 'less than' in raw_data.lower():
		return 0.0
	else:
		return float(re.search('(\d*\.?\d+)', raw_data).group(1))

for col in med_clean_cols:
	read_df[col] = read_df.apply(lambda row : clean_med_cols(row,col), axis=1)


In [654]:
"""
Standardize 'Calcium' to mg
DV of calcium = 1000 mg (100%)
To standardize, multiply percentages by 1000
if 8% => 0.08 * 1000 = 8 * 10
"""
def clean_calcium(row):
	raw_data = row['Calcium']
	if '%' in raw_data:
		return float(re.search('(\d*\.?\d+)', raw_data).group(1)) * 10
	else:
		return float(re.search('(\d*\.?\d+)', raw_data).group(1))

read_df['Calcium'] = read_df.apply(lambda row: clean_calcium(row), axis=1)
read_df['Calcium'].value_counts(dropna=False)

150.0    10
0.0       5
200.0     4
190.0     2
40.0      2
210.0     2
80.0      1
250.0     1
30.0      1
265.0     1
100.0     1
20.0      1
Name: Calcium, dtype: int64

In [655]:
"""
Cleaning 'serving size': only 4 cases of not 28.0 (g)
1. get `indexes` of df that are not 28.0 g serving size
2. index 21 = italian truffle cheese is '1 inch block cube'
	checked website -> 28.0 g (update!)
	remove index 21 from `indexes`
3. Extract number from rest of indexes
"""
read_df['serving size'].value_counts()
indexes = read_df.index[read_df['serving size'] != 28.0].tolist() # step 1
read_df.loc[21, 'serving size'] = 28.0 # step 2
indexes.remove(21)
for index in indexes:
	read_df.loc[index, 'serving size'] = re.search('(\d*\.?\d+)',
	read_df.loc[index, 'serving size']).group(1) # step 3
read_df['serving size'] = read_df['serving size'].astype(float) # to float

In [656]:
# standardize nutrition to 28 g serving
nutrition_cols = ['calories', 'Total Fat', 'Saturated Fat', 'Cholesterol',
'Sodium', 'Total Carbohydrate', 'Protein', 'Calcium']

def standardize_nutrition(row):
    serving = float(row['serving size'])
    for col in nutrition_cols:
        nutr_val = float(row[col]) * (28 / serving)
        row[col] = nutr_val
    return row

read_df.loc[indexes, :] = read_df.loc[indexes, :].apply(lambda row: standardize_nutrition(row), axis = 1)


In [657]:
read_df.to_csv('clean_df.csv', index=False)